<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/04_LDA_Cordis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelado de Tópicos con LDA (Latent Dirichlet Allocation)

## ¿Qué es LDA?
LDA es una técnica de modelado probabilístico que nos permite descubrir "temas" o "tópicos" ocultos en una colección de documentos.
Cada documento se considera una mezcla de varios tópicos, y cada tópico es una distribución sobre palabras.

## ¿Para qué sirve?
- Descubrir temas subyacentes en grandes colecciones de textos
- Clasificar documentos automáticamente
- Recomendar contenidos similares
- Analizar tendencias en documentos

## En este notebook aprenderemos:
1. Preparar y cargar datos textuales
2. Preprocesar texto para LDA
3. Entrenar un modelo LDA
4. Interpretar y evaluar los resultados

## Dataset:
Usaremos una [colección de resúmenes de proyectos europeos](https://docs.google.com/spreadsheets/d/1up1Q9RcWhqQYkeM7b0SiToDJaMYzoSQeG-vfA3wY57s/edit?gid=0#gid=0) extraídos de [CORDIS](https://cordis.europa.eu/projects/en) como ejemplo.
"""

# 1) Importaciones y configuración inicial

In [1]:
# Instalamos las bibliotecas necesarias
!pip install --upgrade -q gspread
!pip install --upgrade -q google-auth-oauthlib

# Instalamos las versiones específicas necesarias
!pip install pyLDAvis==2.1.2

# Reiniciamos el runtime de Colab para asegurar que los cambios surtan efecto
#import IPython
#IPython.Application.instance().kernel.do_shutdown(True)

In [2]:
# Importamos las bibliotecas que usaremos
from sklearn.feature_extraction.text import CountVectorizer  # Para crear bag-of-words
from sklearn.decomposition import LatentDirichletAllocation  # Para el modelo LDA
import numpy as np  # Para operaciones numéricas
import pandas as pd  # Para manipular datos tabulares
import warnings
warnings.filterwarnings('ignore')  # Suprimimos warnings para mayor claridad

# Configuramos la visualización de DataFrames en Colab
from google.colab import data_table
data_table.enable_dataframe_formatter()

print("Bibliotecas cargadas correctamente")

Bibliotecas cargadas correctamente


# 2) Autenticación y carga de datos

Para acceder a los datos, seguiremos estos pasos:

1. Crear una hoja de cálculo en Google Drive:
   - Nombre: 'texts'
   - Columnas: ID, TITLE, DESCRIPTION
   - Primera fila: encabezados
   - Filas siguientes: datos de ejemplo
   
2. Compartir la hoja:
   - Clic en 'Compartir' en Google Sheets
   - Configurar como "Cualquier persona con el enlace puede editar"
   
3. Autenticarnos con Google:

In [14]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Nombre de la hoja de cálculo (crear en Drive con texto de ejemplo)
corpus = 'texts' # texts_nlp
worksheet = gc.open(corpus).sheet1

# Cargar datos a DataFrame
rows = worksheet.get_all_values()
dataset_df = pd.DataFrame.from_records(rows[1:], columns=["ID","TITLE","DESCRIPTION"])

print(f"Datos cargados exitosamente:")
print(f"- Número de documentos: {len(dataset_df)}")
print(f"- Columnas disponibles: {', '.join(dataset_df.columns)}")

# Mostrar los primeros documentos
print("\nPrimeros documentos:")
data_table.DataTable(dataset_df.head(), include_index=False, num_rows_per_page=5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Datos cargados exitosamente:
- Número de documentos: 100
- Columnas disponibles: ID, TITLE, DESCRIPTION

Primeros documentos:


,ID,TITLE,DESCRIPTION
0,EU100000,Visual object population codes relating human...,Two major challenges facing systems neuroscien...
1,EU100001,New Opportunities for Research Funding Agency ...,NORFACE is a co-ordinated common action of fif...
2,EU100002,USA and Europe Cooperation in Mini UAVs,Unmanned Aerial Systems have been an active ar...
3,EU100003,Sustainable Infrastructure for Resilient Urban...,This fellowship aim is to identify how the use...
4,EU100004,Modelling star formation in the local universe,The goal of this proposal is to revolutionize ...


# 3) Procesamiento inicial

Para aplicar LDA, necesitamos convertir nuestros textos en una representación numérica.
Usaremos Bag of Words (BoW), que:
1. Crea un vocabulario con todas las palabras únicas
2. Representa cada documento como un vector de frecuencias de palabras

Parámetros importantes del vectorizador:
- stop_words: palabras comunes a ignorar
- min_df: frecuencia mínima de documento para incluir una palabra
- max_df: frecuencia máxima de documento para incluir una palabra
- lowercase: convertir todo a minúsculas
- token_pattern: patrón para identificar palabras válidas

In [15]:
# Lista de textos a procesar
documents = dataset_df['DESCRIPTION'].tolist()

# Configurar y crear el vectorizador
tf_vectorizer = CountVectorizer(
    stop_words=[],  # No eliminamos stopwords por ahora []
    min_df=1,      # Incluir palabras que aparecen al menos 1 vez
    max_df=1.0,    # Sin límite superior de frecuencia
    lowercase=True, # Convertir todo a minúsculas
    max_features=50000,  # Máximo número de palabras a considerar
    token_pattern='[a-zA-Z0-9]{3,}',  # Palabras de 3+ caracteres
    analyzer = 'word'
)

# Crear la matriz de documentos-términos
bag_of_words = tf_vectorizer.fit_transform(documents)

# Obtener el vocabulario
dictionary = tf_vectorizer.get_feature_names_out()
vocabulary = tf_vectorizer.vocabulary_

print("Estadísticas del preprocesamiento:")
print(f"- Tamaño del vocabulario: {len(dictionary)} palabras únicas")
print(f"- Dimensiones de la matriz: {bag_of_words.shape}")

# Mostrar las palabras más frecuentes
word_freq = bag_of_words.sum(axis=0).A1
top_words_idx = word_freq.argsort()[-10:][::-1]
print("\nPalabras más frecuentes:")
for idx in top_words_idx:
    print(f"- {dictionary[idx]}: {word_freq[idx]} apariciones")

Estadísticas del preprocesamiento:
- Tamaño del vocabulario: 4530 palabras únicas
- Dimensiones de la matriz: (100, 4530)

Palabras más frecuentes:
- the: 1663 apariciones
- and: 1204 apariciones
- will: 379 apariciones
- for: 332 apariciones
- this: 191 apariciones
- that: 169 apariciones
- with: 166 apariciones
- research: 149 apariciones
- are: 149 apariciones
- project: 115 apariciones


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 4) Entrenamiento LDA

El algoritmo LDA tiene varios hiperparámetros importantes:

1. n_topics: Número de tópicos a encontrar
   - Debe elegirse según el conocimiento del dominio
   - Se puede optimizar usando métricas como coherencia o perplejidad

2. alpha: Prior de la distribución documentos-tópicos
   - alpha < 1: documentos se concentran en pocos tópicos
   - alpha > 1: documentos mezclan varios tópicos
   - alpha = 1: distribución uniforme

3. beta: Prior de la distribución tópicos-palabras
   - beta < 1: tópicos más específicos (pocas palabras)
   - beta > 1: tópicos más generales (muchas palabras)
   - beta = 1: distribución uniforme

Probaremos primero con valores conservadores:

In [16]:
# Parámetros del modelo
n_topics = 2    # Número moderado de tópicos para empezar
alpha = 1.0     # Documentos algo especializados
beta = 0.1     # Tópicos bastante específicos

# Crear y entrenar el modelo
print("Configuración del modelo LDA:")
print(f"- Número de tópicos: {n_topics}")
print(f"- Alpha: {alpha}")
print(f"- Beta: {beta}")
print("\nIniciando entrenamiento...\n")

lda = LatentDirichletAllocation(
    n_components=n_topics,      # Número de tópicos
    doc_topic_prior=alpha,      # Prior documentos-tópicos
    topic_word_prior=beta,      # Prior tópicos-palabras
    max_iter=25,               # Máximo de iteraciones
    learning_method='online',   # Método de aprendizaje
    evaluate_every=1,          # Evaluar en cada iteración
    n_jobs=-1,                # Usar todos los cores
    random_state=0,           # Semilla para reproducibilidad
    verbose=1                 # Mostrar progreso
)

# Entrenar el modelo
lda.fit(bag_of_words)

Configuración del modelo LDA:
- Número de tópicos: 2
- Alpha: 1.0
- Beta: 0.1

Iniciando entrenamiento...



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


iteration: 1 of max_iter: 25, perplexity: 2936.7239
iteration: 2 of max_iter: 25, perplexity: 2410.5933
iteration: 3 of max_iter: 25, perplexity: 2238.5408
iteration: 4 of max_iter: 25, perplexity: 2145.6998
iteration: 5 of max_iter: 25, perplexity: 2082.7478
iteration: 6 of max_iter: 25, perplexity: 2034.8295
iteration: 7 of max_iter: 25, perplexity: 1995.9868
iteration: 8 of max_iter: 25, perplexity: 1963.3391
iteration: 9 of max_iter: 25, perplexity: 1935.2936
iteration: 10 of max_iter: 25, perplexity: 1910.8903
iteration: 11 of max_iter: 25, perplexity: 1889.5094
iteration: 12 of max_iter: 25, perplexity: 1870.7148
iteration: 13 of max_iter: 25, perplexity: 1854.1698
iteration: 14 of max_iter: 25, perplexity: 1839.5954
iteration: 15 of max_iter: 25, perplexity: 1826.7522
iteration: 16 of max_iter: 25, perplexity: 1815.4317
iteration: 17 of max_iter: 25, perplexity: 1805.4510
iteration: 18 of max_iter: 25, perplexity: 1796.6497
iteration: 19 of max_iter: 25, perplexity: 1788.8871
it

LatentDirichletAllocation(doc_topic_prior=1.0, evaluate_every=1,
                          learning_method='online', max_iter=25, n_components=2,
                          n_jobs=-1, random_state=0, topic_word_prior=0.1,
                          verbose=1)

# 5) Análisis de Resultados

Analizaremos los resultados desde tres perspectivas:

1. Palabras más relevantes por tópico
2. Documentos más representativos de cada tópico
3. Distribución de tópicos en documentos específicos

In [17]:
# Configuración de visualización
no_top_words = 10      # Número de palabras top por tópico
no_top_documents = 2   # Número de documentos top por tópico

# Obtener las distribuciones
doc_topics = lda.transform(bag_of_words)  # Distribución de tópicos por documento
topics = lda.components_                  # Distribución de palabras por tópico

# 1. Detallar tópicos encontrados
print("TÓPICOS DESCUBIERTOS")
print("Cada tópico se representa por sus palabras más probables\n")

for topic_idx, topic in enumerate(topics):
    print(f" Tópico {topic_idx + 1}:")
    # Obtener índices de las palabras más probables
    top_words_idx = topic.argsort()[:-no_top_words-1:-1]
    top_words = [dictionary[i] for i in top_words_idx]
    top_probs = [topic[i] for i in top_words_idx]

    # Mostrar palabras y sus probabilidades
    for word, prob in zip(top_words, top_probs):
        print(f"   {word}: {prob:.4f}")
    print()

TÓPICOS DESCUBIERTOS
Cada tópico se representa por sus palabras más probables

 Tópico 1:
   the: 1527.7418
   and: 1116.5891
   will: 341.5355
   for: 301.1397
   this: 171.0859
   that: 158.0021
   with: 149.6321
   research: 136.1724
   are: 132.5021
   project: 109.2254

 Tópico 2:
   the: 66.6449
   and: 37.8148
   will: 22.0437
   quantum: 19.6762
   for: 17.3809
   matter: 15.3585
   this: 12.2753
   placebo: 10.6347
   are: 10.6001
   star: 10.3644



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Documentos más representativos por tópico

In [18]:
# 2. Documentos más representativos por tópico
print("\n DOCUMENTOS MÁS REPRESENTATIVOS POR TÓPICO")
print("Se muestran los documentos que más peso tienen en cada tópico\n")

for topic_idx in range(n_topics):
    print(f" Tópico {topic_idx + 1}:")
    # Obtener los documentos más representativos
    top_doc_indices = np.argsort(doc_topics[:,topic_idx])[::-1][:no_top_documents]

    for doc_idx in top_doc_indices:
        title = dataset_df.iloc[doc_idx]['TITLE']
        weight = doc_topics[doc_idx, topic_idx]
        print(f"   '{title}'")
        print(f"      Peso: {weight:.4f}")
    print()


 DOCUMENTOS MÁS REPRESENTATIVOS POR TÓPICO
Se muestran los documentos que más peso tienen en cada tópico

 Tópico 1:
   'A network for supporting the coordination of Supercomputing research between Europe and Latin America'
      Peso: 0.9967
   'Models for Optimising Dynamic Urban Mobility'
      Peso: 0.9965

 Tópico 2:
   'Fundamental Physics at the Low Background Frontier'
      Peso: 0.9844
   'Tailoring Mixed-Metal Chemistry for Frontier Synthetic and Catalytic Applications'
      Peso: 0.9728



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Matriz Documento-Topico

In [19]:
from IPython.display import display, HTML
import pandas as pd
#pd.set_option('display.max_columns', None)

topicnames = ["topic"+ str(x) for x in range(0, lda.n_components)]
norm_doc_topics = []
for i in doc_topics:
  norm_doc_topics.append([ "{0:.3f}".format(weight) for weight in i])

df = pd.DataFrame(norm_doc_topics,
                  columns=topicnames,
                  index=dataset_df['TITLE'].tolist())

data_table.DataTable(df, num_rows_per_page=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic0,topic1
Visual object population codes relating human brains to nonhuman and computational models with representational similarity analysis,0.994,0.006
New Opportunities for Research Funding Agency Co-operation in Europe II,0.993,0.007
USA and Europe Cooperation in Mini UAVs,0.995,0.005
Sustainable Infrastructure for Resilient Urban Environments,0.993,0.007
Modelling star formation in the local universe,0.042,0.958
...,...,...
The Environmental Observation Web and its Service Applications within the Future Internet,0.993,0.007
Future INternet for Smart ENergY,0.995,0.005
Smart Food and Agribusiness: Future Internet for Safe and Healthy Food from Farm to Fork,0.994,0.006
Instant Mobility for Passengers and Goods,0.992,0.008


## Matriz Tópico-Palabra

In [20]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis])

# Assign Column and Index
df_topic_keywords.columns = dictionary
df_topic_keywords.index = topicnames

# View
data_table.DataTable(df_topic_keywords, num_rows_per_page=10)
#df_topic_keywords.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,000,100,146a,1500,1844,192,1993,1mbit,200,2000,...,yes,yet,yield,young,younger,youngest,zebrafish,zone,zones,zoo
topic0,0.000053,0.000091,0.000053,0.000053,0.000053,0.000053,0.000053,0.000053,0.000053,0.000053,...,0.000053,0.000165,0.000099,0.000331,0.000053,0.000053,0.000191,0.000145,0.000007,0.000052
topic1,0.000070,0.001679,0.000073,0.000072,0.000074,0.000074,0.000076,0.000073,0.000075,0.000072,...,0.000070,0.000874,0.000078,0.000081,0.000074,0.000078,0.000073,0.000084,0.000572,0.000077


# 6)  Evaluación del modelo

Usaremos dos métricas principales:

1. Log Likelihood (mayor es mejor):
   - Indica qué tan bien el modelo explica los datos
   - Valores más altos indican mejor ajuste

2. Perplejidad (menor es mejor):
   - Mide qué tan "sorprendido" está el modelo por los datos
   - Valores más bajos indican mejor generalización


In [21]:
# Calcular métricas
log_likelihood = lda.score(bag_of_words)
perplexity = lda.perplexity(bag_of_words)

print("MÉTRICAS DE EVALUACIÓN")
print(f"- Log Likelihood: {log_likelihood:.2f}")
print(f"- Perplejidad: {perplexity:.2f}")

# Comparar con diferentes valores de hiperparámetros
print("\nCOMPARACIÓN DE HIPERPARÁMETROS")
print("Probando diferentes configuraciones para encontrar el mejor modelo...")

# Probar diferentes números de tópicos
n_topics_range = [3, 5, 7, 10]
results = []

for n_top in n_topics_range:
    model = LatentDirichletAllocation(
        n_components=n_top,
        doc_topic_prior=alpha,
        topic_word_prior=beta,
        max_iter=25,
        random_state=0
    )
    model.fit(bag_of_words)

    results.append({
        'n_topics': n_top,
        'perplexity': model.perplexity(bag_of_words),
        'log_likelihood': model.score(bag_of_words)
    })

# Mostrar resultados
results_df = pd.DataFrame(results)
print("\nResultados con diferentes números de tópicos:")
print(results_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


MÉTRICAS DE EVALUACIÓN
- Log Likelihood: -166551.04
- Perplejidad: 1758.13

COMPARACIÓN DE HIPERPARÁMETROS
Probando diferentes configuraciones para encontrar el mejor modelo...

Resultados con diferentes números de tópicos:
   n_topics   perplexity  log_likelihood
0         3  1669.760396  -165401.505510
1         5  1705.509520  -165873.691607
2         7  1752.325459  -166477.301128
3        10  1778.008588  -166801.625695


# 7) Análisis de Similitud

Podemos usar las distribuciones de tópicos para:
1. Encontrar documentos similares
2. Clasificar nuevos documentos
3. Recomendar contenido relacionado

In [22]:
from scipy.spatial.distance import jensenshannon

def get_similar_documents(doc_idx, n=3):
    """Encuentra los documentos más similares a uno dado."""
    # Obtener distribución de tópicos del documento
    doc_topic_dist = doc_topics[doc_idx]

    # Calcular similitud con todos los documentos
    similarities = []
    for i in range(len(doc_topics)):
        if i != doc_idx:
            # Usar distancia Jensen-Shannon
            dist = jensenshannon(doc_topic_dist, doc_topics[i])
            sim = 1 - dist
            similarities.append((i, sim))

    # Ordenar por similitud
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:n]

# Ejemplo con un documento aleatorio
example_doc = np.random.randint(0, len(dataset_df))
print(f" Documento de ejemplo: '{dataset_df.iloc[example_doc]['TITLE']}'")
print("\n Documentos más similares:")

similar_docs = get_similar_documents(example_doc)
for idx, sim in similar_docs:
    print(f"- '{dataset_df.iloc[idx]['TITLE']}'")
    print(f"  Similitud: {sim:.4f}")

 Documento de ejemplo: 'Sustainable Infrastructure for Resilient Urban Environments'

 Documentos más similares:
- 'Multi-scale biomechanical modelling and simulation of the intervertebral disc'
  Similitud: 0.9998
- 'Towards compressive information processing systems'
  Similitud: 0.9997
- 'Neural Circuits Underlying Visually Guided Behaviour'
  Similitud: 0.9996


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 8) Inferencia de Tópicos

In [23]:
text = "Build airplanes in a large-scale" #@param {type:"string"}
vector = tf_vectorizer.transform([text])
topic_dist = lda.transform(vector)[0]

print(f"Texto analizado: '{text}'")
print("Distribución de tópicos:")

# Mostrar cada tópico con su probabilidad
for idx, prob in enumerate(topic_dist, 1):
    bar = "▓" * int(prob * 50)  # Barra visual simple
    print(f"Tópico {idx}: {prob:.2%} {bar}")



Texto analizado: 'Build airplanes in a large-scale'
Distribución de tópicos:
Tópico 1: 61.10% ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
Tópico 2: 38.90% ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
